Source :- https://realpython.com/async-io-python/

Please use Python version > 3.7

Use code below to check:

<code>
from platform import python_version
print(python_version())
</code>

#### Parallelism vs Concurrency vs Threading

<b>Parallelism</b> consists of performing multiple operations at the same time. Multiprocessing is a means to effect parallelism, and it entails spreading tasks over a computer’s central processing units (CPUs, or cores). Multiprocessing is well-suited for CPU-bound tasks: tightly bound for loops and mathematical computations usually fall into this category.

<b>Concurrency</b> is a slightly broader term than parallelism. It suggests that multiple tasks have the ability to run in an overlapping manner. (There’s a saying that concurrency does not imply parallelism.)

<b>Threading</b> is a concurrent execution model whereby multiple threads take turns executing tasks. One process can contain multiple threads. Python has a complicated relationship with threading thanks to its GIL.

<b>Asyncio</b> - async IO is a single-threaded, single-process design: it uses <b> cooperative multitasking </b>. async IO is a style of concurrent programming, but it is not parallelism. It’s more closely aligned with threading than with multiprocessing but is very much distinct from both of these and is a standalone member in concurrency’s bag of tricks.

In [5]:
#Simple example of async function.

import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

In [9]:
import time
s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
Executed in 1.01 seconds.


### The Rules of Async IO
- The syntax async def introduces either a native coroutine or an asynchronous generator.
- The keyword await passes function control back to the event loop. (It suspends the execution of the surrounding coroutine.) If Python encounters an await f() expression in the scope of g(), this is how await tells the event loop, “Suspend execution of g() until whatever I’m waiting on—the result of f()—is returned. In the meantime, go let something else run.”

<code> 
    async def g():
        #Pause here and come back to g() when f() is ready
        r = await f()
        return r
</code>

There’s also a strict set of rules around when and how you can and cannot use async/await.
- A function that you introduce with <code> async def </code> is a <b> coroutine </b>. It may use <code> await, return, or yield </code>, but all of these are optional. Declaring <code> async def noop(): pass </code> is valid.
    1. Using await and/or return creates a <b> coroutine function </b>. To call a coroutine function, you must await it to get its results.
    2. It is less common (and only recently legal in Python) to use <code> yield </code> in an <code> async def </code> block. This creates an <b> asynchronous generator </b>, which you iterate over with <code> async for </code>.
    3. Anything defined with <code> async def </code> may not use <code> yield from </code>, which will raise a <b>SyntaxError</b>.

- Just like it’s a SyntaxError to use yield outside of a def function, it is a SyntaxError to use await outside of an async def coroutine. <b> <i> You can only use await in the body of coroutines. </i> </b>

<b>Examples:</b>

<code>
async def f(x):
    y = await z(x)   # OK - `await` and `return` allowed in coroutines
    return y
 </code>

<code>
async def g(x):
    yield x  # OK - this is an async generator
</code>

<code>
async def m(x):
    yield from gen(x)  # No - SyntaxError
</code>
    
<code>
def m(x):
    y = await z(x)  #Still no - SyntaxError (no `async def` here)
    return y
</code>

Finally, when you use await f(), it’s required that f() be an object that is <b>awaitable</b>. 
Just know that an awaitable object is either:
- another coroutine (or)
- an object defining an .\_\_await\_\_() dunder method that returns an iterator.

In [12]:
import asyncio
import random

# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)

async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f"Initiated makerandom({idx}).")
    i = random.randint(0, 10)
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)
        i = random.randint(0, 10)
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    return i

async def main():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res

random.seed(444)
r1, r2, r3 = await main()
print()
print(f"r1: {r1}, r2: {r2}, r3: {r3}")

Initiated makerandom(0).
makerandom(0) == 4 too low; retrying.
Initiated makerandom(1).
makerandom(1) == 4 too low; retrying.
Initiated makerandom(2).
makerandom(2) == 0 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 7 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 8 too low; retrying.
---> Finished: makerandom(1) == 10
makerandom(0) == 7 too low; retrying.
makerandom(0) == 8 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 7 too low; retrying.
makerandom(0) == 1 too low; retrying.
makerandom(0) == 6 too low; retrying.
---> Finished: makerandom(2) == 9
makerandom(0) == 3 too low; retrying.
makerandom(0) == 9 too low; retrying.
makerandom(0) == 7 too low; retrying.
---> Finished: makerandom(0) == 10

r1: 10, r2: 10, r3: 9


In [4]:
import asyncio
import random
random.seed(444)

async def squares(number):
    print(f"Calculating square of {number}")
    await asyncio.sleep(random.randint(0, 3))
    result = number ** 2
    print(f"Square of {number} -> {result}")
    return result
    
async def main():
    result = await asyncio.gather(squares(1), squares(2), squares(3))
    return result

s1, s2, s3 = await main()
s1, s2, s3

Calculating square of 1
Calculating square of 2
Calculating square of 3
Square of 3 -> 9
Square of 1 -> 1
Square of 2 -> 4


(1, 4, 9)

### Coroutines

source:- https://www.geeksforgeeks.org/coroutine-in-python/

Coroutines are generalizations of subroutines. They are used for <b> cooperative multitasking </b> where a process voluntarily <b> <i> yield </i> </b> (give away) control periodically or when idle in order to enable multiple applications to be run simultaneously. The difference between coroutine and subroutine is : 
 

- Unlike subroutines, coroutines have many entry points for suspending and resuming execution. Coroutine can suspend its execution and transfer control to other coroutine and can resume again execution from the point it left off.
- Unlike subroutines, there is no main function to call coroutines in a particular order and coordinate the results. Coroutines are cooperative that means they link together to form a pipeline. One coroutine may consume input data and send it to other that process it. Finally, there may be a coroutine to display the result.


<b> Coroutine Vs Thread </b>

Now you might be thinking how coroutine is different from threads, both seem to do the same job. 
In the case of threads, it’s an operating system (or run time environment) that switches between threads according to the scheduler. While in the case of a coroutine, it’s the programmer and programming language which decides when to switch coroutines. Coroutines work cooperatively multitask by suspending and resuming at set points by the programmer. 